We first downloaded the Betfair Starting Price (BSP) data for all UK horses in all UK races from 1 March 2013 to 2 May 2024 (See Download Script). The BSP represents the last price (odds) matched on Betfair betting exchange for each horse before the race started and is determined by market supply and demand. This totalled 4’138 csv files, each representing 1 day’s races. It consisted of 113’201 unique races.

In [ ]:
###########################################################################################################################################
#DOWNLOAD SCRIPT
#UK win

date <- as.Date("2023-12-04")

while (date<"2024-05-03") {
  
  form_date <- format(date,"%d%m%Y")
  url<-paste0("https://promo.betfair.com/betfairsp/prices/dwbfpricesukwin",form_date,".csv")
  destfile<-paste0("csv_files/dwbfpricesukwin",form_date,".csv")
  
  skip_to_next <- FALSE
  
  tryCatch(download.file(url, destfile), error = function(e) { skip_to_next <<- TRUE})
  
  if(skip_to_next) { date <- date+1 }  
  
  date <- date+1
  
}
#############################################################################################################################################
#UK place

date <- as.Date("2023-12-04")

while (date<"2024-05-03") {
  
  form_date <- format(date,"%d%m%Y")
  url<-paste0("https://promo.betfair.com/betfairsp/prices/dwbfpricesukplace",form_date,".csv")
  destfile<-paste0("csv_files/dwbfpricesukplace",form_date,".csv")
  
  skip_to_next <- FALSE
  
  tryCatch(download.file(url, destfile), error = function(e) { skip_to_next <<- TRUE})
  
  if(skip_to_next) { date <- date+1 }  
  
  date <- date+1
  
}
############################################################################################################################

These 4’138 csv files were then combined into one data frame (See BSP_Prep.R Script), used to perform an exploratory data analysis (EDA) to develop the hypothesis and to answer the following questions:

1.	How efficient are racing markets i.e. how good predictor are “Win” markets and “Place” respectively of the outcome of horse races?
2.	How does the liquidity between “Win” and “Place” markets compare?
3.	For which race(s) out of the 113’201 should we perform a time series analysis of the trades over the final minutes before the race started?


In [ ]:
#############################################
# Create BSP data frame from 4'138 csv files#
#############################################
#BSP Prep

#Standardise heading name function
convert_to_uppercase <- function(df) {
  colnames(df) <- toupper(colnames(df))
  return(df)
}

#uk WIN

# List files

uk_win_files = list.files(path="csv_files", pattern="*win*.*csv", full.names=TRUE)

# Read and convert files
uk_win_data <- ldply(uk_win_files, function(file) convert_to_uppercase(read_csv(file)))

#uk PLACE

uk_place_files = list.files(path="csv_files", pattern="*place*.*csv", full.names=TRUE)
uk_place_data = ldply(uk_place_files, function(file) convert_to_uppercase(read_csv(file)))

####Clean data and get in right formats

uk_win_data$EVENT_DT<-as.POSIXct(as.character(uk_win_data$EVENT_DT),format="%d-%m-%Y %H:%M")
uk_win_data$EVENT_ID <- as.numeric(as.character(uk_win_data$EVENT_ID))
uk_win_data$SELECTION_ID <- as.numeric(as.character(uk_win_data$SELECTION_ID))
uk_win_data$BSP <- as.numeric(as.character(uk_win_data$BSP))
uk_win_data$PPTRADEDVOL <- as.numeric(as.character(uk_win_data$PPTRADEDVOL))
uk_win_data$PPWAP <- as.numeric(as.character(uk_win_data$PPWAP))
uk_win_data$MORNINGTRADEDVOL <- as.numeric(as.character(uk_win_data$MORNINGTRADEDVOL))
uk_win_data$MORNINGWAP <- as.numeric(as.character(uk_win_data$MORNINGWAP))
uk_win_data$WIN_LOSE <- as.numeric(as.character(uk_win_data$WIN_LOSE))

####

uk_place_data$EVENT_DT<-as.POSIXct(as.character(uk_place_data$EVENT_DT),format="%d-%m-%Y %H:%M")
uk_place_data$EVENT_ID <- as.numeric(as.character(uk_place_data$EVENT_ID))
uk_place_data$SELECTION_ID <- as.numeric(as.character(uk_place_data$SELECTION_ID))
uk_place_data$BSP <- as.numeric(as.character(uk_place_data$BSP))
uk_place_data$PPTRADEDVOL <- as.numeric(as.character(uk_place_data$PPTRADEDVOL))
uk_place_data$PPWAP <- as.numeric(as.character(uk_place_data$PPWAP))
uk_place_data$MORNINGTRADEDVOL <- as.numeric(as.character(uk_place_data$MORNINGTRADEDVOL))
uk_place_data$MORNINGWAP <- as.numeric(as.character(uk_place_data$MORNINGWAP))
uk_place_data$WIN_LOSE <- as.numeric(as.character(uk_place_data$WIN_LOSE))

#Adding Some Fields
data_csv_uk_win <- uk_win_data

data_csv_uk_win$Implied_Perc_BSP<-1/data_csv_uk_win$BSP
data_csv_uk_win$Implied_Perc_PPWAP<-1/data_csv_uk_win$PPWAP

data_csv_uk_win$Unique_ID <- paste0(data_csv_uk_win$EVENT_DT,data_csv_uk_win$MENU_HINT)
Uk_win_horse_count <-  data_csv_uk_win%>% group_by(Unique_ID) %>% count(Unique_ID)
data_csv_uk_win <- merge(data_csv_uk_win,Uk_win_horse_count,by = c("Unique_ID", "Unique_ID"))
data_csv_uk_win$Unique_ID2<-paste0(data_csv_uk_win$Unique_ID,data_csv_uk_win$SELECTION_ID)

data_csv_uk_place <- uk_place_data

data_csv_uk_place$Implied_Perc_BSP<-1/data_csv_uk_place$BSP
data_csv_uk_place$Implied_Perc_PPWAP<-1/data_csv_uk_place$PPWAP

data_csv_uk_place$Unique_ID <- paste0(data_csv_uk_place$EVENT_DT,data_csv_uk_place$MENU_HINT)
Uk_place_horse_count <-  data_csv_uk_place%>% group_by(Unique_ID) %>% count(Unique_ID)
data_csv_uk_place <- merge(data_csv_uk_place,Uk_place_horse_count,by = c("Unique_ID", "Unique_ID"))
data_csv_uk_place$Unique_ID2<-paste0(data_csv_uk_place$Unique_ID,data_csv_uk_place$SELECTION_ID)

All_uk<-merge(x=data_csv_uk_win, y=data_csv_uk_place, by = c("Unique_ID2","n"))


##This step is to check how man horses Won the race and how many were Placed
Result_Counts <- All_uk %>% 
  group_by(Unique_ID.x) %>% 
  summarise(Win_perc_Sum_BSP = (sum(Implied_Perc_BSP.x)),Place_perc_Sum_BSP = (sum(Implied_Perc_BSP.y)),
            Win_perc_Sum_PPWAP = (sum(Implied_Perc_PPWAP.x)),Place_perc_Sum_PPWAP = (sum(Implied_Perc_PPWAP.y)),
            Win_Count=sum(WIN_LOSE.x),Place_count=sum(WIN_LOSE.y))

Result_Counts <- na.omit(Result_Counts)

All_Uk <- merge(x=All_uk,y=Result_Counts, by=c("Unique_ID.x"))

##We remove the races where 2 horses Won (this is a very rare event when the photo finish cannot even determine the winning horse)
All_Uk <- filter(All_Uk,Win_Count<2)


##We develop a data frame for the hypothesis formation analysis
BSP_DF <- All_Uk[,c("n","EVENT_ID.x" ,"EVENT_NAME.x","EVENT_DT.x","SELECTION_NAME.x","SELECTION_ID.x","WIN_LOSE.x","BSP.x", "Implied_Perc_BSP.x","PPTRADEDVOL.x",
                    "Place_count","BSP.y","Implied_Perc_BSP.y", "PPTRADEDVOL.y")]

write.csv(BSP_DF, file = "BSP_DF.csv", row.names = FALSE)

The BSP_DF is used in hypothesis_formation_BSP_Pred_Acc.R to measure the Efficiency and liquidity of Place and Win markets.